In [1]:
import sys

sys.path.append("../../")

from components.modeling import keywordExtractor
import pandas as pd

extractor = keywordExtractor(dir="../../data/preprocess/eng_han.csv")


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
test_df = pd.read_parquet("modeling_test.parquet")
test_series = test_df.iloc[0]


In [3]:
test_result = dict()


#### extract_keyword_list 메서드 결과값 저장


In [4]:
test_result["extract_keyword_list"] = extractor.extract_keyword_list(test_series)

test_result["_convert_series_to_keyword_list"] = extractor._convert_series_to_keyword_list(
    test_series
)

test_result["_extract_keywords"] = extractor._extract_keywords(
    test_result["_convert_series_to_keyword_list"]
)

test_result["_map_english_to_hangeul"] = extractor._map_english_to_hangeul(
    test_result["_extract_keywords"]
)


#### create_keyword_embedding 메서드 결과값 저장


In [5]:
test_result["create_keyword_embedding"] = extractor.create_keyword_embedding(test_series)

test_result["tokenize_keyword"] = extractor.tokenize_keyword(test_result["extract_keyword_list"])

test_result["_create_keyword_embedding"] = extractor._create_keyword_embedding(
    test_result["tokenize_keyword"]
)


#### create_doc_embeddings 메서드 결과값 저장


In [6]:
test_result["create_doc_embeddings"] = extractor.create_doc_embeddings(test_series)

test_result["_convert_series_to_str"] = extractor._convert_series_to_str(test_series)

test_result["tokenize_doc"] = extractor.tokenize_keyword(test_result["_convert_series_to_str"])

test_result["_create_doc_embeddings"] = extractor._create_doc_embeddings(
    test_result["tokenize_doc"]
)


#### extract_keyword 메서드 결과값 저장


In [7]:
test_result["extract_keyword"] = extractor.extract_keyword(test_df)

test_result["_calc_cosine_similarity"] = extractor._calc_cosine_similarity(
    test_result["create_doc_embeddings"], test_result["create_keyword_embedding"]
)

test_result["_filter_top_n_keyword"] = extractor._filter_top_n_keyword(
    test_result["extract_keyword_list"], test_result["_calc_cosine_similarity"].flatten()
)


In [8]:
test_result["extract_keyword"]

{'isbn13': '9791127459321',
 'keywords': ['머신러닝',
  '템플릿',
  'sqlalchemy',
  '과정',
  '플라스크',
  'api',
  '라우팅',
  '도커',
  'pytest',
  'config',
  '파이썬',
  '알고리즘',
  '코드',
  '로지스틱',
  '실행',
  '삭제',
  '준비',
  'uri',
  '하기',
  '조작']}

In [9]:
import pickle


def write_pkl(item: str, dir: str = None):
    # store list in binary file so 'wb' mode
    with open(f"{dir}", "wb") as fp:
        pickle.dump(item, fp)
        print("Done writing list into a binary file")


write_pkl(item=test_result, dir="modeling_test_result")


Done writing list into a binary file
